In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/detect-fake-text-kerasnlp-tf-torch-jax-infer/__results__.html
/kaggle/input/detect-fake-text-kerasnlp-tf-torch-jax-infer/submission.csv
/kaggle/input/detect-fake-text-kerasnlp-tf-torch-jax-infer/__notebook__.ipynb
/kaggle/input/detect-fake-text-kerasnlp-tf-torch-jax-infer/__output__.json
/kaggle/input/detect-fake-text-kerasnlp-tf-torch-jax-infer/custom.css
/kaggle/input/keras-nlp-deberta-v3-base-en-vocab-ds/vocab.spm
/kaggle/input/llm-science-exam-lib-ds/keras_core-0.1.7-py3-none-any.whl
/kaggle/input/llm-science-exam-lib-ds/keras_nlp-0.6.2-py3-none-any.whl
/kaggle/input/daigt-kerasnlp-ckpt/fold0.keras
/kaggle/input/daigt-kerasnlp-ckpt/fold1.keras
/kaggle/input/daigt-kerasnlp-ckpt/fold2.keras
/kaggle/input/daigt-kerasnlp-ckpt/model.png
/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays

In [19]:
!pip install /kaggle/input/llm-science-exam-lib-ds/keras_core-0.1.7-py3-none-any.whl --no-deps
!pip install /kaggle/input/llm-science-exam-lib-ds/keras_nlp-0.6.2-py3-none-any.whl --no-deps

Processing /kaggle/input/llm-science-exam-lib-ds/keras_core-0.1.7-py3-none-any.whl
keras-core is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/llm-science-exam-lib-ds/keras_nlp-0.6.2-py3-none-any.whl
keras-nlp is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [20]:
import os
os.environ["KERAS_BACKEND"] = "torch"

import keras_nlp
import keras_core as keras 
import keras_core.backend as K
import jax
import tensorflow as tf

import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt

from glob import glob
from tqdm.notebook import tqdm
import gc

In [21]:
print("TensorFlow:", tf.__version__)
print("Keras:", keras.__version__)
print("KerasNLP:", keras_nlp.__version__)

TensorFlow: 2.15.0
Keras: 0.1.7
KerasNLP: 0.6.2


In [22]:
class CFG:
    verbose = 0  # Verbosity
    device = 'GPU'  # Device
    seed = 42  # Random seed
    batch_size = 6  # Batch size
    drop_remainder = True  # Drop incomplete batches
    ckpt_dir = "/kaggle/input/daigt-kerasnlp-ckpt"  # Name of pretrained models
    sequence_length = 200  # Input sequence length
    class_names = ['real','fake']  # Class names [A, B, C, D, E]
    num_classes = len(class_names)  # Number of classes
    class_labels = list(range(num_classes))  # Class labels [0, 1, 2, 3, 4]
    label2name = dict(zip(class_labels, class_names))  # Label to class name mapping
    name2label = {v: k for k, v in label2name.items()}  # Class name to label mapping

In [23]:
keras.utils.set_random_seed(CFG.seed)

In [24]:
def get_device():
    "Detect and intializes GPU/TPU automatically"
    try:
        # Connect to TPU
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() 
        # Set TPU strategy
        strategy = tf.distribute.TPUStrategy(tpu)
        print(f'> Running on TPU', tpu.master(), end=' | ')
        print('Num of TPUs: ', strategy.num_replicas_in_sync)
        device=CFG.device
    except:
        # If TPU is not available, detect GPUs
        gpus = tf.config.list_logical_devices('GPU')
        ngpu = len(gpus)
         # Check number of GPUs
        if ngpu:
            # Set GPU strategy
            strategy = tf.distribute.MirroredStrategy(gpus) # single-GPU or multi-GPU
            # Print GPU details
            print("> Running on GPU", end=' | ')
            print("Num of GPUs: ", ngpu)
            device='GPU'
        else:
            # If no GPUs are available, use CPU
            print("> Running on CPU")
            strategy = tf.distribute.get_strategy()
            device='CPU'
    return strategy, device

In [25]:
strategy, CFG.device = get_device()
CFG.replicas = strategy.num_replicas_in_sync

> Running on GPU | Num of GPUs:  2


In [26]:
BASE_PATH = '/kaggle/input/llm-detect-ai-generated-text'

In [27]:
test_df = pd.read_csv(f'{BASE_PATH}/test_essays.csv')  # Read CSV file into a DataFrame

# Display information about the train data
print("# Test Data: {:,}".format(len(test_df)))
print("# Sample:")
display(test_df.head(2))

# Test Data: 3
# Sample:


,id,prompt_id,text
0,0000aaaa,2,Aaa bbb ccc.
1,1111bbbb,3,Bbb ccc ddd.


In [28]:
vocab_path = '/kaggle/input/keras-nlp-deberta-v3-base-en-vocab-ds/vocab.spm'
tokenizer= keras_nlp.models.DebertaV3Tokenizer(vocab_path)
preprocessor= keras_nlp.models.DebertaV3Preprocessor(tokenizer, sequence_length=CFG.sequence_length)


In [29]:
outs = preprocessor(test_df.text.iloc[0])  # Process options for the first row

for k, v in outs.items():
    print(k, ":", v.shape)

token_ids : torch.Size([200])
padding_mask : torch.Size([200])


In [30]:
def preprocess_fn(text, label=None):
    text = preprocessor(text)  # Preprocess text
    return (text, label) if label is not None else text  # Return processed text and label if available

In [31]:
#Pipeline
def build_dataset(texts, labels=None, batch_size=32,
                  cache=False, drop_remainder=True,
                  augment=False, repeat=False, shuffle=1024):
    AUTO = tf.data.AUTOTUNE  # AUTOTUNE option
    slices = (texts,) if labels is None else (texts, labels)  # Create slices
    ds = tf.data.Dataset.from_tensor_slices(slices)  # Create dataset from slices
    ds = ds.cache() if cache else ds  # Cache dataset if enabled
    ds = ds.map(preprocess_fn, num_parallel_calls=AUTO)  # Map preprocessing function
    ds = ds.repeat() if repeat else ds  # Repeat dataset if enabled
    opt = tf.data.Options()  # Create dataset options
    if shuffle: 
        ds = ds.shuffle(shuffle, seed=CFG.seed)  # Shuffle dataset if enabled
        opt.experimental_deterministic = False
    ds = ds.with_options(opt)  # Set dataset options
    ds = ds.batch(batch_size, drop_remainder=drop_remainder)  # Batch dataset
    ds = ds.prefetch(AUTO)  # Prefetch next batch
    return ds  # Return the built dataset

In [32]:
#FetchDate
def get_test_dataset(test_df):
    test_texts = test_df.text.tolist()  # Extract testation texts
    
    # Build testation dataset
    test_ds = build_dataset(test_texts, labels=None,
                             batch_size=min(CFG.batch_size*CFG.replicas, len(test_df)), cache=False,
                             shuffle=False, drop_remainder=False, repeat=False)
    
    return test_ds  # Return datasets and dataframes

In [33]:
def build_model():
    # Create a DebertaV3Classifier model
    classifier = keras_nlp.models.DebertaV3Classifier.from_preset(
        CFG.preset,
        load_weights=False,
        preprocessor=None,
        num_classes=1 # one output per one option, for five options total 5 outputs
    )
    inputs = classifier.input
    logits = classifier(inputs)
        
    # Compute final output
    outputs = keras.layers.Activation("sigmoid")(logits)
    model = keras.Model(inputs, outputs)
    return model


In [34]:
# Get the checkpoint directory and name
ckpt_dir = CFG.ckpt_dir
ckpt_name = ckpt_dir.split('/')[3]

# Copy the checkpoints to a new directory in the /kaggle directory
!cp -r {ckpt_dir} /kaggle/{ckpt_name}

# List all the checkpoint paths in the new directory
new_ckpt_dir = f"/kaggle/{ckpt_name}"
ckpt_paths = glob(os.path.join(new_ckpt_dir, '*.keras'))

print("Total CKPT:", len(ckpt_paths))

Total CKPT: 3


In [36]:
#Prediction (Inference)
fold_preds = np.zeros(shape = (len(test_df),), dtype = 'float32')

for ckpt_path in tqdm(ckpt_paths):
    model = keras.models.load_model(
        ckpt_path, 
        compile =False, )
    test_ds = get_test_dataset(test_df)
    preds = model.predict(
        test_ds,
        batch_size = min(CFG.batch_size * CFG.replicas * 2, len(test_df)),
        verbose = 1
    )
    
    fold_preds += preds.squeeze()/len(ckpt_paths)
    del model
    gc.collect()
    
    


  0%|          | 0/3 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/keras_core/src/trainers/trainer.py:166: UserWarning: `jit_compile` is not yet enabled for the PyTorch backend. Proceeding with `jit_compile=False`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/keras_core/src/saving/serialization_lib.py:713: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step


In [37]:
# Format predictions and true answers
pred_answers = (fold_preds > 0.5).astype(int).squeeze()

# Check 5 Predictions
print("# Predictions\n")
for i in range(3):
    row = test_df.iloc[i]
    text  = row.text
    pred_answer = CFG.label2name[pred_answers[i]]
    print(f"❓ Text {i+1}:\n{text}\n")
    print(f"🤖 Predicted: {pred_answer}\n")
    print("-"*90, "\n")

# Predictions

❓ Text 1:
Aaa bbb ccc.

🤖 Predicted: fake

------------------------------------------------------------------------------------------ 

❓ Text 2:
Bbb ccc ddd.

🤖 Predicted: fake

------------------------------------------------------------------------------------------ 

❓ Text 3:
CCC ddd eee.

🤖 Predicted: fake

------------------------------------------------------------------------------------------ 

